## 8.1 Transfer Learning Faster R-CNN

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%208/8.1%20transfer_learning_faster_rcnn.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu.png?raw=1" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu-success.png?raw=1" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

#### 8.1.1 Download Dataset From Roboflow
- Folow instruction in [4.1 dataset_annotation_roboflow.ipynb](https://github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%204/84.1%20dataset_annotation_roboflow.ipynb) to prepare `Scissors Dataset` example,
- Open `Roboflow` > `Project` > `Versions` menu
- Then click `Download Dataset`<br>
<img src="resource/rb-download-dataset.png" width="850px">
- Choose `COCO` format and select `Show download code` then click `Continue` <br>
<img src="resource/rb-download-format.png" width="350px">
- click `Copy` icon to copy roboflow download code<br>
<img src="resource/rb-copy-download-code.png" width="350px">
- Then <font color="orange">replace below code</font> using the copied roboflow download code above,


In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="xxxxxxxxxxxxxxxxxx")
# project = rf.workspace("xxxxxxxxxxxxx").project("xxxxxxxxxxxxxxx")
# version = project.version(1)
# dataset = version.download("coco")

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import json
import os

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # Load all image files and annotations
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.annotations = json.load(open(os.path.join(root, "annotations.json")))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        # Get annotations for this image
        ann = self.annotations[idx]
        boxes = torch.as_tensor(ann["boxes"], dtype=torch.float32)
        labels = torch.as_tensor(ann["labels"], dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}
        if self.transforms:
            img = self.transforms(img)
        return img, target

    def __len__(self):
        return len(self.imgs)


In [ ]:
transform = T.Compose([
    T.ToTensor(),
    # Add other augmentations as needed
])


In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2

# Load the pre-trained model
model = fasterrcnn_resnet50_fpn_v2(weights='DEFAULT')
num_classes = 2  # Change this based on your dataset (including background)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torch.nn.Linear(in_features, num_classes)


In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.ops import box_iou

# Assume dataset and DataLoader for training and validation have been set up
dataset_train = CustomDataset(root="path/to/train", transforms=transform)
dataset_val = CustomDataset(root="path/to/val", transforms=transform)
data_loader_train = DataLoader(dataset_train, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
data_loader_val = DataLoader(dataset_val, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()
    iou_thresholds = [0.5, 0.75]  # Set thresholds for mAP calculation
    ap_all = []
    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            for i, output in enumerate(outputs):
                pred_boxes = output['boxes']
                pred_scores = output['scores']
                gt_boxes = targets[i]['boxes'].to(device)
                
                # Compute IoU for each prediction and ground truth box
                ious = box_iou(pred_boxes, gt_boxes)
                matched = (ious > iou_thresholds[0]).sum().item()
                
                # Calculate AP per threshold
                for iou_th in iou_thresholds:
                    tp = (ious > iou_th).sum().item()
                    fp = len(pred_boxes) - tp
                    fn = len(gt_boxes) - tp
                    precision = tp / (tp + fp + 1e-6)
                    recall = tp / (tp + fn + 1e-6)
                    ap_all.append(precision * recall)  # Simplified AP

    mAP = sum(ap_all) / len(ap_all)
    return mAP

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, targets in data_loader_train:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        total_loss += losses.item()

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    # Validation step
    mAP = evaluate(model, data_loader_val, device)
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader_train)}, mAP: {mAP:.4f}")
